# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data_weatherpy/cities.csv")
weather_df = weather_data
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bambous Virieux,-20.3428,57.7575,76.21,57,40,9.22,MU,1629958981
1,La Santisima Trinidad,-14.8333,-64.9000,80.83,34,51,3.40,BO,1629958982
2,Qaanaaq,77.4840,-69.3632,37.04,94,100,8.05,GL,1629958982
3,Haverfordwest,51.8017,-4.9691,60.55,89,3,4.25,GB,1629958982
4,Port Alfred,-33.5906,26.8910,73.96,35,48,9.80,ZA,1629958800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure the gmaps to use locations as lat long, and humidity as they weight
gmaps.configure(g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]


fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities with wind speed less than 20 mph, cloudiness less than 5 and max temp between 70 and 90
narrowed_weather_df = weather_data.loc[(weather_df["Wind Speed"] <= 20) & (weather_df["Cloudiness"] <=5) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 90)].dropna()

narrowed_weather_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Awjilah,29.1081,21.2869,80.02,42,0,7.20,LY,1629958984
11,Faya,18.3851,42.4509,78.91,29,2,13.82,SA,1629958985
45,East London,-33.0153,27.9116,72.54,28,0,5.75,ZA,1629958995
52,Serik,36.9169,31.0989,89.85,29,0,2.13,TR,1629958997
66,Saint-Pierre,-21.3393,55.4781,76.64,62,0,17.27,RE,1629959001
...,...,...,...,...,...,...,...,...,...
529,Farah,32.5000,62.5000,86.56,11,0,9.33,AF,1629959145
544,Chipinge,-20.1883,32.6236,70.09,37,0,2.75,ZW,1629959149
548,Nānākuli,21.3906,-158.1547,80.55,65,1,6.91,US,1629959151
552,‘Ewa Beach,21.3156,-158.0072,80.55,65,1,6.91,US,1629958894


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = pd.DataFrame(narrowed_weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
9,Awjilah,LY,29.1081,21.2869,
11,Faya,SA,18.3851,42.4509,
45,East London,ZA,-33.0153,27.9116,
52,Serik,TR,36.9169,31.0989,
66,Saint-Pierre,RE,-21.3393,55.4781,


In [6]:
print('-------------------------')
print("Beginning Data Retrieval ")
print('-------------------------')



params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

#URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Loop to match the city names with the list of cities 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #Add keywords
    params["location"] = f"{lat},{lng}"
    #Make the API call   
    response = requests.get(base_url, params=params).json()
    
    #Get the results
    results = response['results'] 
    #If the hotel name is found print it and append hotel_df file with the hotel name
    #If not available then print an error message
    
    
    try:
        print(f"The hotel in {city_name} is {results[0]['name']}.")

        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Error:Missing field")
             
print("-------------------------------")
print("Data Retrieval Complete")
print("-------------------------------")


-------------------------
Beginning Data Retrieval 
-------------------------
Error:Missing field
Error:Missing field
The hotel in East London is Southern Sun Hemingways.
Error:Missing field
The hotel in Saint-Pierre is Le Battant Des Lames.
Error:Missing field
The hotel in Quatre Cocos is Constance Belle Mare Plage Mauritius.
The hotel in Bethel is Golden Hill Inn Danbury.
Error:Missing field
The hotel in Opelousas is Holiday Inn & Suites Opelousas, an IHG Hotel.
The hotel in Laurel is Sleep Inn & Suites.
Error:Missing field
Error:Missing field
The hotel in Bījār is Hotel Bame Iran.
The hotel in Strezhevoy is Гостиница "Турист".
The hotel in Abhā is Shafa Abha Hotel.
The hotel in Waddān is بشير التاجوري.
The hotel in Shetpe is Gostinitsa Shetpe.
The hotel in Al Bardīyah is Hotel Burdy.
Error:Missing field
The hotel in Karabük is SAFRANBOLU "LEYLA HANIM KONAĞI".
The hotel in Gold Coast is Novotel Surfers Paradise.
Error:Missing field
The hotel in Yulara is Sails in the Desert.
The hote

In [7]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
9,Awjilah,LY,29.1081,21.2869,
11,Faya,SA,18.3851,42.4509,
45,East London,ZA,-33.0153,27.9116,Southern Sun Hemingways
52,Serik,TR,36.9169,31.0989,
66,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))